In [108]:
!pip install -q pyinterval
!pip install -q pandas

In [109]:
from interval import interval
from interval import imath
import pandas as pd
import numpy as np
import math

In [110]:
def width(x_i):
    return sum([x.sup - x.inf for x in x_i])

def midpoint(x_i):
    return (x_i[0].sup + x_i[0].inf) / 2

def itos(x_i):
    return f"[{x_i[0].inf:9.7f}, {x_i[0].sup:9.7f}]"

#### *Метод поділу відрізка навпіл*

In [111]:
def bisection(f, start, end, tol=0.01, iteration=1):
    X = interval[start, end]
    F = f(X)
    # немає кореня на проміжку якщо немає нуля
    if 0 not in F:
        print(iteration, 'немає кореня                    ', itos(X), itos(F), sep='\t')
        iteration +=1
        return
    # знайдено корінь на проміжку, який задовольняє окіл
    if width(X) < tol:
        print(iteration, 'корінь знайдено                 ', itos(X), itos(F), sep='\t')
        iteration +=1
        return
    
    print(iteration, 'переходимо до наступної ітерації', itos(X), itos(F), sep='\t')
    mid = midpoint(X)
    bisection(f, start, mid, tol, iteration + 1)
    bisection(f, mid, end, tol, iteration + 1)

#### *Приклад 1*

In [112]:
func = lambda x: (x**3)+4*(x**2)+x-6
bisection(func, 0, 2.2, 0.01)

1	переходимо до наступної ітерації	[0.0000000, 2.2000000]	[-6.0000000, 26.2080000]
2	переходимо до наступної ітерації	[0.0000000, 1.1000000]	[-6.0000000, 1.2710000]
3	немає кореня                    	[0.0000000, 0.5500000]	[-6.0000000, -4.0736250]
3	переходимо до наступної ітерації	[0.5500000, 1.1000000]	[-4.0736250, 1.2710000]
4	немає кореня                    	[0.5500000, 0.8250000]	[-4.0736250, -1.8909844]
4	переходимо до наступної ітерації	[0.8250000, 1.1000000]	[-1.8909844, 1.2710000]
5	немає кореня                    	[0.8250000, 0.9625000]	[-1.8909844, -0.4402090]
5	переходимо до наступної ітерації	[0.9625000, 1.1000000]	[-0.4402090, 1.2710000]
6	переходимо до наступної ітерації	[0.9625000, 1.0312500]	[-0.4402090, 0.3818665]
7	немає кореня                    	[0.9625000, 0.9968750]	[-0.4402090, -0.0374317]
7	переходимо до наступної ітерації	[0.9968750, 1.0312500]	[-0.0374317, 0.3818665]
8	переходимо до наступної ітерації	[0.9968750, 1.0140625]	[-0.0374317, 0.1701371]
9	корінь зн

#### *Метод Мура для р-ня*

In [113]:
def moore(f, df, start, end, tol=1e-6, iteration=1):
    X = interval([start, end])
    X_width = width(X)

    # [Step 1]: Stop if no 0 in F(X)
    if 0 not in f(X):
        print(iteration, 'кореня немає', itos(X), X_width, sep='\t')
        return
    
    # Return result if tolerance is satisfied
    if X_width < tol:
        print(iteration, 'корінь знайдено', itos(X), X_width, sep='\t')
        return
    
    # [Step 2]: Half if 0 in F'(X)
    x_mid = midpoint(X)
    df_X = df(X)
    if 0 in df_X:
        print(iteration, 'робимо поділ', itos(X), X_width, sep='\t')
        moore(f, df, start, x_mid, tol, iteration + 1)
        moore(f, df, x_mid,   end, tol, iteration + 1)
        return
    
    # [Step 3]: Stop if X_next is empty
    f_mid = f(x_mid)
    U = x_mid - f_mid / df_X
    X_next = U & X
    if not X_next:
        print(iteration, 'кореня немає', itos(U) + ' & ' + itos(X), sep='\t')
        return

    # [Step 4]: Continue with narrowed interval
    print(iteration, 'продовжуємо', itos(X), X_width, sep='\t')
    moore(f, df, X_next[0].inf, X_next[0].sup, tol, iteration + 1)

#### *Приклад 1*

In [114]:
func = lambda x: (x**3)+4*(x**2)+x-6
dfunc = lambda x: 3*(x**2)+8*x+1 
moore(func, dfunc, -5, -2.9, 0.0001)

1	робимо поділ	[-5.0000000, -2.9000000]	2.1
2	продовжуємо	[-5.0000000, -3.9500000]	1.0499999999999998
3	кореня немає	[-4.0248385, -3.9500000]	0.07483847128378418
2	робимо поділ	[-3.9500000, -2.9000000]	1.0500000000000003
3	кореня немає	[-3.4210180, -2.5031837] & [-3.9500000, -3.4250000]
3	робимо поділ	[-3.4250000, -2.9000000]	0.5249999999999999
4	кореня немає	[-3.1439326, -2.8410048] & [-3.4250000, -3.1625000]
4	продовжуємо	[-3.1625000, -2.9000000]	0.26249999999999973
5	продовжуємо	[-3.0146034, -2.9000000]	0.11460344840531
6	продовжуємо	[-3.0146034, -2.9892471]	0.02535639483494112
7	продовжуємо	[-3.0001504, -2.9998332]	0.00031723503495850736
8	корінь знайдено	[-3.0000000, -3.0000000]	1.684310868554917e-08


#### *Приклад 2*

In [115]:
func = lambda x: (x**3)+4*(x**2)+x-6
dfunc = lambda x: 3*(x**2)+8*x+1 
moore(func, dfunc, 0, 7, 0.0001)

1	продовжуємо	[0.0000000, 7.0000000]	7.0
2	продовжуємо	[0.0000000, 3.0618873]	3.061887254901961
3	продовжуємо	[0.0000000, 1.3725285]	1.3725285476442508
4	продовжуємо	[0.8624636, 1.3725285]	0.510064965986315
5	продовжуємо	[0.9686286, 1.0319564]	0.06332784567340966
6	корінь знайдено	[0.9999889, 1.0000105]	2.1620662314703054e-05


### Метод Хансена для р-ня

In [116]:
def hansen(f, df, start, end, tol=1e-6, iteration=1):
    X = interval([start, end])
    X_width = width(X)

    # [Step 1]: Stop if no 0 in F(X)
    if 0 not in f(X):
        print(iteration, 'кореня немає', itos(X), X_width, sep='\t')
        return

    # Return result if tolerance is satisfied
    if X_width < tol:
        print(iteration, "корінь знайдено", itos(X), X_width, sep='\t')
        return
    
    # [Step 2]: Half if 0 in F'(X)
    x_mid = midpoint(X)
    f_mid = f(x_mid)
    df_X = df(X)
    if f_mid == 0.0:
        print(iteration, "корінь знайдено", itos(X), X_width, '<====== { f == 0 }', sep='\t')
        hansen(f, df, start, x_mid, tol, iteration + 1)
        hansen(f, df, x_mid, end, tol, iteration + 1)
        return
    
    # [Step 3]: Stop if X_next is empty
    U = x_mid - f_mid / df_X
    X_next = U & X
    if not X_next:
        print(iteration, 'кореня немає', itos(U) + '&' + itos(X), sep='\t')
        return

    # [Step 4]: Continue with narrowed intervals
    print(iteration, "продовжуємо", itos(X), X_width, sep='\t')
    [hansen(f, df, x.inf, x.sup, tol, iteration + 1) for x in X_next]

#### *Приклад 1*

In [117]:
func = lambda x: (x**3)+4*(x**2)+x-6
dfunc = lambda x: 3*(x**2)+8*x+1 

hansen(func, dfunc, 0, 7, 0.0000001)

1	продовжуємо	[0.0000000, 7.0000000]	7.0
2	продовжуємо	[0.0000000, 3.0618873]	3.061887254901961
3	продовжуємо	[0.0000000, 1.3725285]	1.3725285476442508
4	продовжуємо	[0.8624636, 1.3725285]	0.510064965986315
5	продовжуємо	[0.9686286, 1.0319564]	0.06332784567340966
6	продовжуємо	[0.9999889, 1.0000105]	2.1620662314703054e-05
7	корінь знайдено	[1.0000000, 1.0000000]	6.962208587424357e-12


#### *Приклад 2*

In [118]:
func = lambda x: (x**3)+4*(x**2)+x-6
dfunc = lambda x: 3*(x**2)+8*x+1 

hansen(func, dfunc, -5, -2.9, 0.0000001)

1	продовжуємо	[-5.0000000, -2.9000000]	2.1
2	кореня немає	[-5.0000000, -4.6159314]	0.384068627450981
2	продовжуємо	[-3.7763281, -2.9000000]	0.8763281250000006
3	продовжуємо	[-3.2427844, -2.9000000]	0.3427843502164212
4	продовжуємо	[-3.0380747, -2.9000000]	0.13807470210730433
5	продовжуємо	[-3.0308877, -2.9907302]	0.04015756631931655
6	продовжуємо	[-3.0013504, -2.9985899]	0.0027605651967199663
7	продовжуємо	[-3.0000003, -2.9999997]	5.35428501979851e-07
8	корінь знайдено	[-3.0000000, -3.0000000]	1.2878587085651816e-14


#### *Метод Кравчика для р-ня*

In [119]:
def kravchyk(f, df, start, end, tol=1e-6, iteration=1):
    X = interval([start, end])
    X_width = width(X)

    # [Step 1]: Stop if no 0 in F(X)
    if 0 not in f(X):
        print(iteration, "кореня немає", itos(X), X_width, sep='\t')
        return

    # Return result if tolerance is satisfied
    if X_width < tol:
        print(iteration, "корінь знайдено", itos(X), X_width, sep='\t')
        return

    # Split interval if 0 in F'(X)
    x_mid = midpoint(X)
    df_X = df(X)
    if 0 in df_X:
        print(iteration, "продовжуємо", itos(X), X_width, sep='\t')
        kravchyk(f, df, start, x_mid, tol, iteration + 1)
        kravchyk(f, df, x_mid,   end, tol, iteration + 1)
        return

    # Stop if X_next is empty
    df_x = df(x_mid)
    K = x_mid - f(x_mid) / df_x + (1 - df(X) / df_x)*(X - x_mid)
    X_next = K & X
    if not X_next:
        print(iteration, "кореня немає", itos(K) + '&' + itos(X), sep='\t')
        return

    # Display interval if shrunk or not
    print(iteration, "*", itos(X), X_width, sep='\t')
    kravchyk(f, df, X_next[0].inf, X_next[0].sup, tol, iteration + 1)

#### *Приклад 1*

In [120]:
func = lambda x: (x**3)+4*(x**2)+x-6
dfunc = lambda x: 3*(x**2)+8*x+1 

kravchyk(func, dfunc, -5, -2.9)

1	продовжуємо	[-5.0000000, -2.9000000]	2.1
2	*	[-5.0000000, -3.9500000]	1.0499999999999998
3	кореня немає	[-4.0814572, -3.9500000]	0.13145723611008098
2	продовжуємо	[-3.9500000, -2.9000000]	1.0500000000000003
3	кореня немає	[-3.4183894, -3.0718394]&[-3.9500000, -3.4250000]
3	продовжуємо	[-3.4250000, -2.9000000]	0.5249999999999999
4	кореня немає	[-3.1343977, -2.9995977]&[-3.4250000, -3.1625000]
4	*	[-3.1625000, -2.9000000]	0.26249999999999973
5	*	[-3.1072541, -2.9000000]	0.20725407331975587
6	*	[-3.0700741, -2.9299586]	0.14011547868158925
7	*	[-3.0321593, -2.9678407]	0.06431851924067544
8	*	[-3.0067474, -2.9932526]	0.013494723247821305
9	*	[-3.0002962, -2.9997038]	0.000592310335064461
10	*	[-3.0000006, -2.9999994]	1.140241445263257e-06
11	корінь знайдено	[-3.0000000, -3.0000000]	4.225952920933196e-12


#### *Приклад 2*

In [121]:
func = lambda x: (x**3)+4*(x**2)+x-6
dfunc = lambda x: 3*(x**2)+8*x+1 

kravchyk(func, dfunc, -2.9, -0.1)

1	продовжуємо	[-2.9000000, -0.1000000]	2.8
2	продовжуємо	[-2.9000000, -1.5000000]	1.4
3	продовжуємо	[-2.9000000, -2.2000000]	0.6999999999999997
4	продовжуємо	[-2.9000000, -2.5500000]	0.3500000000000001
5	кореня немає	[-3.1591848, -2.9866121]&[-2.9000000, -2.7250000]
5	продовжуємо	[-2.7250000, -2.5500000]	0.17499999999999982
6	кореня немає	[-3.4381911, -3.3552125]&[-2.7250000, -2.6375000]
6	продовжуємо	[-2.6375000, -2.5500000]	0.08749999999999991
7	кореня немає	[-4.0617686, -4.0238385]&[-2.6375000, -2.5937500]
7	продовжуємо	[-2.5937500, -2.5500000]	0.04375000000000018
8	кореня немає	[-2.5937500, -2.5718750]	0.02187500000000009
8	кореня немає	[-2.5718750, -2.5500000]	0.02187500000000009
4	продовжуємо	[-2.5500000, -2.2000000]	0.34999999999999964
5	продовжуємо	[-2.5500000, -2.3750000]	0.17499999999999982
6	продовжуємо	[-2.5500000, -2.4625000]	0.08749999999999991
7	продовжуємо	[-2.5500000, -2.5062500]	0.04375000000000018
8	кореня немає	[-2.5500000, -2.5281250]	0.02187500000000009
8	кореня н

## **Система**

In [122]:
def print_interval(iteration, status, X, Y):
    print(iteration, status, 'X:'+itos(X), 'Y:'+itos(Y), sep='\t')

# returns determinant, 
def det2x2(m):
    return m[0][0]*m[1][1] - m[0][1]*m[1][0]

# subinterval, t - top, b - bottom, l - left, r - right
def half_interval(x):
    mid = midpoint(x)
    return (interval[x[0].inf, mid], interval[mid, x[0].sup])

#### Метод Мура для системи

In [123]:
def moore_system(f, df, intervals, tol=1e-6, iteration=1):
    X, Y = intervals
    X_width, Y_width = width(X), width(Y)

    # [Step 1]: Stop if no 0 in F(X, Y)
    f_XY = f(X, Y)
    if 0 not in f_XY[0] or 0 not in f_XY[1]:
        print_interval(iteration, 'немає кореня', X, Y)
        return

    # Return result if tolerance is satisfied
    if max(X_width, Y_width) < tol:
        print_interval(iteration, 'корінь знайдено', X, Y)
        return

    # [Step 2]: Half if 0 in F'(X, Y)
    x_mid, y_mid = midpoint(X), midpoint(Y)
    df_XY = df(X, Y)
    df_XY_det = interval(det2x2(df_XY))

    if 0 in df_XY_det:
        print_interval(iteration, 'поділ', X, Y)
        Xl, Xr = half_interval(X)
        Yl, Yr = half_interval(Y)
        if X_width < tol: # half only Y
            moore_system(f, df, (X, Yl), tol, iteration + 1)
            moore_system(f, df, (X, Yr), tol, iteration + 1)
        elif Y_width < tol: # half only X
            moore_system(f, df, (Xl, Y), tol, iteration + 1)
            moore_system(f, df, (Xr, Y), tol, iteration + 1)
        else: # half everything
            moore_system(f, df, (Xl, Yl), tol, iteration + 1)
            moore_system(f, df, (Xl, Yr), tol, iteration + 1)
            moore_system(f, df, (Xr, Yl), tol, iteration + 1)
            moore_system(f, df, (Xr, Yr), tol, iteration + 1)
        return

    f1m, f2m = f(x_mid, y_mid)
    df1x, df1y, df2x, df2y = *df_XY[0], *df_XY[1]
    det = df_XY_det
    U_X = x_mid + ( (-df2y/det)*f1m + ( df1y/det)*f2m )
    U_Y = y_mid + ( ( df2x/det)*f1m + (-df1x/det)*f2m )
    X_next, Y_next = U_X & X, U_Y & Y
    
    # [Step 3]: Stop if X_next or Y_next is empty
    if not X_next or not Y_next:
        print('empty ==> ', end='')
        print_interval(iteration, 'кореня немає', X, Y)
        return

    # [Step 4]: Continue with narrowed interval
    print_interval(iteration, "продовжуємо", X, Y)
    moore_system(f, df, (X_next, Y_next), tol, iteration + 1)

#### Приклад

In [124]:
func = lambda x, y: (x**2 + y**2 - 17,
                     x*y-4)
dfunc = lambda x, y: ((2*x, 2*y),
                      (y, x))
init_intervals = (interval[0, 2.3], interval[3, 5.9])
moore_system(func, dfunc, init_intervals, tol=0.0001)

1	продовжуємо	X:[0.0000000, 2.3000000]	Y:[3.0000000, 5.9000000]
2	продовжуємо	X:[0.0000000, 2.3000000]	Y:[3.0000000, 4.9650391]
3	продовжуємо	X:[0.3739316, 1.1361422]	Y:[3.8600919, 4.3308919]
4	продовжуємо	X:[0.9467005, 1.0582299]	Y:[3.9651141, 4.0416855]
5	продовжуємо	X:[0.9997781, 1.0002222]	Y:[3.9997646, 4.0002334]
6	корінь знайдено	X:[1.0000000, 1.0000000]	Y:[4.0000000, 4.0000000]


#### Метод Хансена для системи

In [125]:
def hansen_system(f, df, intervals, tol=1e-6, iteration=1):
    X, Y = intervals
    X_width, Y_width = width(X), width(Y)    
    
    # Stop if no 0 in F(X, Y)
    f_XY = f(X, Y)
    if 0 not in f_XY[0] or 0 not in f_XY[1]:
        print_interval(iteration, 'кореня нема', X, Y)
        return
    
    # Return result if tolerance is satisfied
    if max(X_width, Y_width) < tol:
        print_interval(iteration, 'корінь знайдено', X, Y)
        return

    x_mid, y_mid = midpoint(X), midpoint(Y)
    df_XY = df(X, Y)
    df_XY_det = interval(det2x2(df_XY))  
    f1m, f2m = f(x_mid, y_mid)

    # Check midpoints
    if f1m == 0.0 and f2m == 0.0:
        print('поділ', end=' ')
        print_interval(iteration, 'корінь знайдено', X, Y)
        Xl, Xr = half_interval(X)
        Yl, Yr = half_interval(Y)
        hansen_system(f, df, (Xl, Yl), tol, iteration + 1)
        hansen_system(f, df, (Xl, Yr), tol, iteration + 1)
        hansen_system(f, df, (Xr, Yl), tol, iteration + 1)
        hansen_system(f, df, (Xr, Yr), tol, iteration + 1)
        return
 
    df1x, df1y, df2x, df2y = *df_XY[0], *df_XY[1]
    det = df_XY_det
    U_X = x_mid + ( (-df2y/det)*f1m + ( df1y/det)*f2m )
    U_Y = y_mid + ( ( df2x/det)*f1m + (-df1x/det)*f2m )
    X_next, Y_next = U_X & X, U_Y & Y

    # Stop if X_next is empty
    if not X_next or not Y_next:
        print('empty ==>', end=' ')
        print_interval(iteration, 'кореня нема', X, Y)
        return

    # Force split if X and Y haven't changed
    if X == X_next and Y == Y_next:
        print_interval(iteration, 'поділ', X, Y)
        if(X_width > Y_width):
            Xl, Xr = half_interval(X)
            hansen_system(f, df, (Xl, Y), tol, iteration + 1)
            hansen_system(f, df, (Xr, Y), tol, iteration + 1)
        else:
            Yl, Yr = half_interval(Y)
            hansen_system(f, df, (X, Yl), tol, iteration + 1)
            hansen_system(f, df, (X, Yr), tol, iteration + 1)
        return
    
    # Continue with narrowed intervals
    print_interval(iteration, "продовжуємо", X_next, Y_next)
    for Xi in X_next:
        for Yi in Y_next:
            hansen_system(f, df, (interval(Xi), interval(Yi)), tol, iteration + 1)

#### Приклад

In [126]:
func = lambda x, y: (x**2 + y**2 - 17,
                     x*y-4)
dfunc = lambda x, y: ((2*x, 2*y),
                      (y, x))
init_intervals = (interval[0, 2.3], interval[3, 5.9])
hansen_system(func, dfunc, init_intervals, tol=1e-6)

1	продовжуємо	X:[0.0000000, 2.3000000]	Y:[3.0000000, 4.9650391]
2	продовжуємо	X:[0.3739316, 1.1361422]	Y:[3.8600919, 4.3308919]
3	продовжуємо	X:[0.9467005, 1.0582299]	Y:[3.9651141, 4.0416855]
4	продовжуємо	X:[0.9997781, 1.0002222]	Y:[3.9997646, 4.0002334]
5	продовжуємо	X:[1.0000000, 1.0000000]	Y:[4.0000000, 4.0000000]
6	корінь знайдено	X:[1.0000000, 1.0000000]	Y:[4.0000000, 4.0000000]


#### Метод Кравчика для системи рівнянь

In [127]:
def kravchyk_system(f, df, intervals, tol=1e-6, iteration=1):
    X, Y = intervals
    X_width, Y_width = width(X), width(Y)

    # Stop if no 0 in F(X, Y)
    f_XY = f(X, Y)
    if 0 not in f_XY[0] or 0 not in f_XY[1]:
        print_interval(iteration, '-', X, Y)
        return

    # Return result if tolerance is satisfied
    if max(X_width, Y_width) < tol:
        print_interval(iteration, '+', X, Y)
        return

    # Half if 0 in F'(X, Y)
    x_mid, y_mid = midpoint(X), midpoint(Y)
    df_XY = df(X, Y)
    df_XY_det = interval(det2x2(df_XY))

    if 0 in df_XY_det:
        Xl, Xr = half_interval(X)
        Yl, Yr = half_interval(Y)
        if X_width < tol: # half only Y
            print_interval(iteration, '**', X, Y)
            kravchyk_system(f, df, (X, Yl), tol, iteration + 1)
            kravchyk_system(f, df, (X, Yr), tol, iteration + 1)
        elif Y_width < tol: # half only X
            print_interval(iteration, '**', X, Y)
            kravchyk_system(f, df, (Xl, Y), tol, iteration + 1)
            kravchyk_system(f, df, (Xr, Y), tol, iteration + 1)
        else: # half everything
            print_interval(iteration, '****', X, Y)
            kravchyk_system(f, df, (Xl, Yl), tol, iteration + 1)
            kravchyk_system(f, df, (Xl, Yr), tol, iteration + 1)
            kravchyk_system(f, df, (Xr, Yl), tol, iteration + 1)
            kravchyk_system(f, df, (Xr, Yr), tol, iteration + 1)
        return

    f1m, f2m = f(x_mid, y_mid)
    df_mid = df(x_mid, y_mid)
    df1x,   df1y,  df2x,  df2y =  *df_XY[0], *df_XY[1]
    df1xm, df1ym, df2xm, df2ym = *df_mid[0], *df_mid[1]
    det = det2x2(df_mid)
    retard_X = (1 + df2x*df1ym/det - df1x*df2ym/det)*(X-x_mid) \
             + (    df2y*df1ym/det - df1y*df2ym/det)*(Y-y_mid)
    retard_Y = (  - df2x*df1xm/det + df1x*df2xm/det)*(X-x_mid) \
             + (1 - df2y*df1xm/det + df1y*df2xm/det)*(Y-y_mid)
    K_X = x_mid + ( (-df2ym/det)*f1m + ( df1ym/det)*f2m ) + retard_X
    K_Y = y_mid + ( ( df2xm/det)*f1m + (-df1xm/det)*f2m ) + retard_Y
    X_next, Y_next = K_X & X, K_Y & Y
    
    # Stop if X_next or Y_next is empty
    if not X_next or not Y_next:
        print('empty ==> ', end='')
        print_interval(iteration, '-', X, Y)
        return

    # Continue with narrowed interval
    print_interval(iteration, "*", X, Y)
    kravchyk_system(f, df, (X_next, Y_next), tol, iteration + 1)

#### Приклад

In [128]:
func = lambda x, y: (x**2 + y**2 - 17,
                     x*y-4)
dfunc = lambda x, y: ((2*x, 2*y),
                      (y, x))
init_intervals = (interval[0, 2.3], interval[3, 5.9])
kravchyk_system(func, dfunc, init_intervals, tol=1e-6)

1	*	X:[0.0000000, 2.3000000]	Y:[3.0000000, 5.9000000]
2	*	X:[0.0000000, 2.0254600]	Y:[3.0000000, 5.0551677]
3	*	X:[0.3096891, 1.6904355]	Y:[3.3096819, 4.6905159]
4	*	X:[0.6822403, 1.3177597]	Y:[3.6822403, 4.3177597]
5	*	X:[0.9326858, 1.0673142]	Y:[3.9326858, 4.0673142]
6	*	X:[0.9969792, 1.0030208]	Y:[3.9969792, 4.0030208]
7	*	X:[0.9999939, 1.0000061]	Y:[3.9999939, 4.0000061]
8	+	X:[1.0000000, 1.0000000]	Y:[4.0000000, 4.0000000]
